In [1]:
from sage.interfaces.gap import  get_gap_memory_pool_size, set_gap_memory_pool_size
set_gap_memory_pool_size(24364842180)

W = WeylGroup("E8", prefix="s")
[s1,s2,s3,s4,s5,s6,s7,s8] = W.simple_reflections()

##################################################################################

n = rank(W)
w0 = W.long_element()
e = W(1)


####### Kazhdan-Lusztig polynomials ##########################

# A faster implementation of KL-polynomials (using the optional package Coxeter 3) is given by this
# Fokko Ducloux’s Coxeter3 C++ library.

# Had to install it: I just typed "sage -i coxeter3" in the terminal.

# It seems that one can direcly coerce from WeylGroup to CoxeterGroup and vice versa.
# I will therefore use CoxeterGroup to calculate KL-polynomials, but for all other Bruhat business I will use WeylGroup.

R.<q> = LaurentPolynomialRing(QQ)

KL = KazhdanLusztigPolynomial(W,q)  # KL-polynomials implemented in standard Sage way
# http://doc.sagemath.org/html/en/reference/combinat/sage/combinat/kazhdan_lusztig.html


CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")

def KLP(x,y):
    '''Returns the KL-polynomial, implemented in "Coxeter3" package by Fokko du Cloux.
    http://math.univ-lyon1.fr/~ducloux/coxeter/coxeter3/english/coxeter3_e.html'''
    
    if x not in W:
        x = convert_from_123(x)
    if y not in W:
        y = convert_from_123(y)
    
    return CoxeterPackage.kazhdan_lusztig_polynomial(CoxeterPackage(x), CoxeterPackage(y))
    # If "coxeter3" is not installed, remove the line 'CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")'
    # and in this function return KL.P(x,y)

#Point:
#    - standard Sage way: KL.P(x,y)
#    - faster way: KLP(x,y) 


def mu(w,x):
    '''Returns the KL mu-function with arguments w,x.
    By Humphrey's BGG book p. 175 and p. 169, for w<x we have:
    mu(x,w) = mu(w0*w,w0*x),
    mu(w,x) = dim Exit^1 (L_w,L_x) = dim Exit^1(L_x,L_w) = dim Exit^1(Delta_x,L_w).'''

    if w not in W:
        w = convert_from_123(w)
    if x not in W:
        x = convert_from_123(x)
        
    if w.bruhat_le(x):
        poly_dict = KLP(w,x).dict()       
        j = (x.length()-w.length()-1)/2 
        if j not in poly_dict.keys():
            return 0
        return poly_dict[j]

    return 0


def convert_to_123(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == W(1):
        return "e"
    
    return str(w).replace("s","").replace("*","")


def convert_from_123(string):
    '''Converts one element from W in the "123" string notation to the usual "s1*s2*s3" notation.'''
    
    if type(string)== Integer:
        string = str(string)
        
    if string == "e":
        return W(1)
    
    string = "*".join([char for char in string])
    
    for i in range(1,n+1):
        string = string.replace(str(i),"s%s"%i)
    
    return eval(string)


def DR(w):
    '''Returns the set of simple right descents of w.'''
    
    return {W.simple_reflections()[i] for i in w.descents()}


def DL(w):
    '''Returns the set of simple left   descents of w.'''
        
    return DR(w.inverse())


def AL(w):
    '''Returns the set of simple left ascends of w.'''
    
    DescLe = list(DL(w))
    AscLe = [s for s in W.simple_reflections() if s not in DescLe]
    return set(AscLe)


def AR(w):
    '''Returns the set of simple right ascends of w.'''
    
    DescRi = list(DR(w))
    AscRi = [s for s in W.simple_reflections() if s not in DescRi]
    return set(AscRi)



####################################

def coeff_one(i):
    if i != 1:
        return str(i)
    else:
        return ""
    
def KLPv_e(w):
    p = KLP(e,w)
    l = w.length()
    pv = sum(p.coefficients()[i] * q^(l - 2*p.exponents()[i]) for i in range(len(p.exponents())))
    
    # Format pv nicely for latex:
    
    return " + ".join(reversed([ coeff_one(pv.coefficients()[i])+"v^{%s}"%pv.exponents()[i] for i in range(len(pv.exponents())) ]))






DynkinDiagram(W)

        O 2
        |
        |
O---O---O---O---O---O---O
1   3   4   5   6   7   8   
E8

In [2]:
# This is only for E8

small = [ convert_from_123(str(x)) for x in [1, 13, 134, 1342, 1345, 13456, 134567, 1345678,
 2, 24, 243, 245, 2456, 24567, 245678,
 3, 34, 345, 3456, 34567, 345678,
 4, 45, 456, 4567, 45678,
 5, 56, 567, 5678,
 6, 67, 678,
 7, 78,
 8]] + [ convert_from_123(str(x)).inverse() for x in [13, 134, 1342, 1345, 13456, 134567, 1345678,
 24, 243, 245, 2456, 24567, 245678,
 34, 345, 3456, 34567, 345678,
 45, 456, 4567, 45678,
 56, 567, 5678,
 67, 678,
 78]]

penultimate = [ x*w0 for x in small ]



def join(i,j):
    return eval(str(i)+str(j))

def sep(x):
    return (eval(str(x)[0]), eval(str(x)[1]))


# Dictionary: w[11], w[12], ...

w = dict()
for x in penultimate:
    w[join(eval(convert_to_123(list(AL(x))[0])),eval(convert_to_123(list(AR(x))[0])))] = x

# Transfer operations from W to ij format:

elements = []
for i in range(1,9):
    for j in range(1,9):
        elements.append(join(i,j))

def bruhat_le(x,y):
    return ( w[x] ).bruhat_le( w[y] )

def length(x):
    return(w[x]).length()

# To make it faster:
lengths = {11: 119,  12: 116,  13: 118,  14: 117,  15: 116,  16: 115,  17: 114,  18: 113,  21: 116,  22: 119, 
           23: 117,  24: 118,  25: 117,  26: 116,  27: 115,  28: 114,  31: 118,  32: 117,  33: 119,  34: 118, 
           35: 117,  36: 116,  37: 115,  38: 114,  41: 117,  42: 118,  43: 118,  44: 119,  45: 118,  46: 117, 
           47: 116,  48: 115,  51: 116,  52: 117,  53: 117,  54: 118,  55: 119,  56: 118,  57: 117,  58: 116, 
           61: 115,  62: 116,  63: 116,  64: 117,  65: 118,  66: 119,  67: 118,  68: 117,  71: 114,  72: 115, 
           73: 115,  74: 116,  75: 117,  76: 118,  77: 119,  78: 118,  81: 113,  82: 114,  83: 114,  84: 115, 
           85: 116,  86: 117,  87: 118,  88: 119}


# Construct J as a directed graph (not very fast)

J = DiGraph([elements,[(12, 14),  (12, 32),  (13, 11),  (13, 33),  (14, 13),  (14, 34),  (15, 14),  (15, 35),  (16, 15),  
                       (16, 36),  (17, 16),  (17, 37),  (18, 17),  (18, 38),  (21, 23),  (21, 41),  (23, 24),  (23, 43),  
                       (24, 22),  (24, 44),  (25, 24),  (25, 45),  (26, 25),  (26, 46),  (27, 26),  (27, 47),  (28, 27),  
                       (28, 48),  (31, 11),  (31, 33),  (32, 34),  (32, 42),  (34, 33),  (34, 44),  (35, 34),  (35, 45),  
                       (36, 35),  (36, 46),  (37, 36),  (37, 47),  (38, 37),  (38, 48),  (41, 31),  (41, 43),  (42, 22),  
                       (42, 44),  (43, 33),  (43, 44),  (45, 44),  (45, 55),  (46, 45),  (46, 56),  (47, 46),  (47, 57),  
                       (48, 47),  (48, 58),  (51, 41),  (51, 53),  (52, 42),  (52, 54),  (53, 43),  (53, 54),  (54, 44),  
                       (54, 55),  (56, 55),  (56, 66),  (57, 56),  (57, 67),  (58, 57),  (58, 68),  (61, 51),  (61, 63),  
                       (62, 52),  (62, 64),  (63, 53),  (63, 64),  (64, 54),  (64, 65),  (65, 55),  (65, 66),  (67, 66),  
                       (67, 77),  (68, 67),  (68, 78),  (71, 61),  (71, 73),  (72, 62),  (72, 74),  (73, 63),  (73, 74),  
                       (74, 64),  (74, 75),  (75, 65),  (75, 76),  (76, 66),  (76, 77),  (78, 77),  (78, 88),  (81, 71),  
                       (81, 83),  (82, 72),  (82, 84),  (83, 73),  (83, 84),  (84, 74),  (84, 85),  (85, 75),  (85, 86),  
                       (86, 76),  (86, 87),  (87, 77),  (87, 88)]])


# Construct J from scratch (10-20 minutes?)

#J = DiGraph()
#for x in elements:
#    lx = length(x)
#    for y in elements:
#        if length(y) == lx+1:
#            if bruhat_le(x,y):
#                J.add_edge([x,y])



In [3]:
# Penultimate polynomials


R.<v> = LaurentPolynomialRing(QQ)

p = dict()


# manual entry:
p[18] = R(v^113 + v^107 + v^103 + v^97) 


# 8. column
p[38] = (v+v^-1)*p[18]
p[48] = (v+v^-1)*p[38] - p[18]
p[28] = p[48]/(v+v^-1)
p[58] = (v+v^-1)*p[48] - p[28]- p[38]
p[68] = (v+v^-1)*p[58] - p[48]
p[78] = (v+v^-1)*p[68] - p[58]
p[88] = (v+v^-1)*p[78] - p[68]

# 8. row:
for i in range(1,9):
    p[80 + i] = p[i*10 + 8]

# 7. column
p[77] = (v+v^-1)*p[87]

# 6. column
p[76] = (v+v^-1)*p[86]
p[67] = p[76]
p[66] = (v+v^-1)*p[76] - p[86]

# 5. column 
p[75] = (v+v^-1)*p[85]
p[57] = p[75]
p[65] = (v+v^-1)*p[75] - p[85]
p[56] = p[65]
p[55] = (v+v^-1)*p[65] - p[75]

# 4. column
p[74] = (v+v^-1)*p[84]
p[47] = p[74]
p[64] = (v+v^-1)*p[74] - p[84]
p[46] = p [64]
p[54] = (v+v^-1)*p[64] - p[74]
p[45] = p [54]
p[44] = (v+v^-1)*p[54] - p[64]

#3. column
p[73] = (v+v^-1)*p[83]
p[37] = p[73]
p[63] = (v+v^-1)*p[73] - p[83]
p[36] = p[63]
p[53] = (v+v^-1)*p[63] - p[73]
p[35] = p[53]
p[43] = (v+v^-1)*p[53] - p[63]
p[34] = p[43]

#2. column
p[72] = (v+v^-1)*p[82]
p[27] = p[72]
p[62] = (v+v^-1)*p[72] - p[82]
p[26] = p[62]
p[52] = (v+v^-1)*p[62] - p[72]
p[25] = p[52]
p[42] = (v+v^-1)*p[52] - p[62]
p[24] = p[42]

#1. column
p[71] = (v+v^-1)*p[81]
p[17] = p[71]
p[61] = (v+v^-1)*p[71] - p[81]
p[16] = p[61]
p[51] = (v+v^-1)*p[61] - p[71]
p[15] = p[51]
p[41] = (v+v^-1)*p[51] - p[61]
p[14] = p[41]

# 1. 3x3 minor
p[23] = p[43]/(v+v^-1)
p[32] = p[23]
p[12] = p[32]/(v+v^-1)
p[21] = p[12]
p[33] = (v+v^-1)*p[43] - p[53] - p[23]
p[13] = p[33]/(v+v^-1)
p[31] = p[13]
p[22] = (v+v^-1)*p[42] - p[32] - p[52]
p[11] = (v+v^-1)*p[31] - p[41]

In [8]:
for x in elements:
    i,j = sep(x)
    print("p[(%s,%s)]=%s" %(i,j, p[x] )  )

p[(1,1)]=v^119 + v^113 + v^109 + v^107 + v^103 + v^101 + v^97 + v^91
p[(1,2)]=v^116 + v^112 + v^110 + v^108 + v^106 + v^104 + v^102 + v^100 + v^98 + v^94
p[(1,3)]=v^118 + v^114 + v^112 + v^110 + 2*v^108 + v^106 + v^104 + 2*v^102 + v^100 + v^98 + v^96 + v^92
p[(1,4)]=v^93 + v^95 + v^97 + 2*v^99 + 2*v^101 + 2*v^103 + 2*v^105 + 2*v^107 + 2*v^109 + 2*v^111 + v^113 + v^115 + v^117
p[(1,5)]=v^94 + v^96 + v^98 + 2*v^100 + v^102 + 2*v^104 + 2*v^106 + v^108 + 2*v^110 + v^112 + v^114 + v^116
p[(1,6)]=v^95 + v^97 + v^99 + v^101 + v^103 + 2*v^105 + v^107 + v^109 + v^111 + v^113 + v^115
p[(1,7)]=v^96 + v^98 + v^102 + v^104 + v^106 + v^108 + v^112 + v^114
p[(1,8)]=v^97 + v^103 + v^107 + v^113
p[(2,1)]=v^116 + v^112 + v^110 + v^108 + v^106 + v^104 + v^102 + v^100 + v^98 + v^94
p[(2,2)]=v^119 + v^115 + v^113 + v^111 + 2*v^109 + v^107 + 2*v^105 + v^103 + 2*v^101 + v^99 + v^97 + v^95 + v^91
p[(2,3)]=v^117 + v^115 + v^113 + 2*v^111 + 2*v^109 + 2*v^107 + 2*v^105 + 2*v^103 + 2*v^101 + 2*v^99 + v^97 + v^95 

In [5]:
def ver_out(x):
    i,j = sep(x)
    result = []
    for k in range(1,9):
        y = join(k,j)
        if (x,y,None) in J.edges():
            result.append(y)
    return result[0]   # only one element in the result, but error in the diagonal case

def ver_in(x):
    i,j = sep(x)
    result = []
    for k in range(1,9):
        y = join(k,j)
        if (y,x,None) in J.edges():
            result.append(y)
    return result

def hor_out(x):
    i,j = sep(x)
    result = []
    for k in range(1,9):
        y = join(i,k)
        if (x,y,None) in J.edges():
            result.append(y)
    return result[0] # only one element in the result, but error in the diagonal case

def hor_in(x):
    i,j = sep(x)
    result = []
    for k in range(1,9):
        y = join(i,k)
        if (y,x,None) in J.edges():
            result.append(y)
    return result



def check_criteria_1(x):
    i,j = sep(x)
    pol = p[x]
    
    # is it a Laurent polynomial?
    if pol not in R:
#        print("p[%s] not a Laurent polynomial"%x)
        return False        
    
    pol = R(pol)
    # is it a polynomial?
    for m in R(pol).exponents():
        if m < 0:
#            print("p[%s] not a polynomial"%x)
            return False

    # with non-negative integral coefficients?
    for c in pol.coefficients():
        if c not in ZZ or c<0:
#            print("p[%s] does not have non-negative integer coefficients"%x)
            return False
    
    # leading term = 1*v^length ?
    if pol.exponents()[-1] != lengths[x] or pol.coefficients()[-1] != 1:
#        print("p[%s] does not have the correct leading term"%x)
        return False        
    
    # all terms > a, or >=a for diagonal ?
    m = pol.exponents()[0]
    if i != j and not m>91:
#        print("p[%s] has a too low monomial degree"%x)
        return False
    
    if i == j and m != 91:
#        print("p[%s] has a too low monomial degree"%x)
        return False   
    
    return True



def check_criteria_2(x):
    # Assuming check_criteria() has been checked FOR ALL x.

    i,j = sep(x)
    pol = R(p[x])
    
    # if not diagonal, check the Lemma
    if i != j:
        if (v+v^-1)*p[x] !=  p[ver_out(x)] + sum(p[y] for y in  ver_in(x) ):
#            print("Lemma does not hold for %s"%x)
            return False       
    
    # if not diagonal, check the opposite version of the Lemma
    if i != j:
        if (v+v^-1)*p[x] !=  p[hor_out(x)] + sum(p[y] for y in  hor_in(x) ):
#            print("Lemma does not hold for %s"%x)
            return False  

    # if diagonal, check whether p_{i,ii} has non-negative coefficients
    if i==j:
        pol_i_ii = R(v^120 + sum(p[y] for y in  ver_in( x )) - v*pol)
        for c in pol_i_ii.coefficients():
            if c<0:
#                print("p_%s,%s does not have non-negative coefficients"%(i,x))
                return False
                         
    # if diagonal, check whether p[ii] - v*p_{i,ii} has non-negative coefficients    
        for c in (pol - v*pol_i_ii).coefficients():
            if c<0:
#                print("p[%s] - v*p_{%s,%s} does not have non-negative coefficients"%(x,i,x))
                return False

    return True

In [6]:
assume = True

for x in elements:
    if not check_criteria_1(x):
        assume = False

if assume == True:
    for x in elements:
        if not check_criteria_2(x):
            assume = False

assume

True

In [9]:
# LaTeX code for the Bruhat Graph of J for E8


def edges(x):
    return [ sep(edge[1]) for edge in J.edges() if edge[0]==x]

def directions(x):
    dom = sep(x)
    return [(cod[0]-dom[0], cod[1]-dom[1]) for cod in edges(x)]

def arrow(pair):
    x, y = pair
    direction = ""
    if x>0:
        direction += "d"*x
    if x<0:
        direction += "u"*abs(x)
    if y>0:
        direction += "r"*y
    if y<0:
        direction += "l"*abs(y)
        
    bend = ""
    if direction in ["rr", "uu"]:
        bend = "@/_1pc/"
    
    if direction in ["ll", "dd"]:
        bend = "@/^1pc/"
    
    return "\\ar%s[%s]"%(bend,direction)


print(" \\\\\n".join([ " & ".join(["w_{%s%s} %s"%(i, j, " ".join([arrow(pair) for pair in directions(join(i,j))]) ) for j in range(1,9)])  for i in range(1,9)]))
        


w_{11}  & w_{12} \ar@/_1pc/[rr] \ar@/^1pc/[dd] & w_{13} \ar@/^1pc/[ll] \ar@/^1pc/[dd] & w_{14} \ar[l] \ar@/^1pc/[dd] & w_{15} \ar[l] \ar@/^1pc/[dd] & w_{16} \ar[l] \ar@/^1pc/[dd] & w_{17} \ar[l] \ar@/^1pc/[dd] & w_{18} \ar[l] \ar@/^1pc/[dd] \\
w_{21} \ar@/_1pc/[rr] \ar@/^1pc/[dd] & w_{22}  & w_{23} \ar[r] \ar@/^1pc/[dd] & w_{24} \ar@/^1pc/[ll] \ar@/^1pc/[dd] & w_{25} \ar[l] \ar@/^1pc/[dd] & w_{26} \ar[l] \ar@/^1pc/[dd] & w_{27} \ar[l] \ar@/^1pc/[dd] & w_{28} \ar[l] \ar@/^1pc/[dd] \\
w_{31} \ar@/_1pc/[uu] \ar@/_1pc/[rr] & w_{32} \ar@/_1pc/[rr] \ar[d] & w_{33}  & w_{34} \ar[l] \ar[d] & w_{35} \ar[l] \ar[d] & w_{36} \ar[l] \ar[d] & w_{37} \ar[l] \ar[d] & w_{38} \ar[l] \ar[d] \\
w_{41} \ar[u] \ar@/_1pc/[rr] & w_{42} \ar@/_1pc/[uu] \ar@/_1pc/[rr] & w_{43} \ar[u] \ar[r] & w_{44}  & w_{45} \ar[l] \ar[d] & w_{46} \ar[l] \ar[d] & w_{47} \ar[l] \ar[d] & w_{48} \ar[l] \ar[d] \\
w_{51} \ar[u] \ar@/_1pc/[rr] & w_{52} \ar[u] \ar@/_1pc/[rr] & w_{53} \ar[u] \ar[r] & w_{54} \ar[u] \ar[r] & w_{55}  & w_

In [20]:
# Table of KL for the penultimate E8


def nice_print(pol):  # reverse monomials if they are not decreasing.
    pol = R(pol)
    deg = max(pol.exponents())
    str_pol = str(latex(pol))
    is_deg = str_pol[3:6]
    
    if is_deg[-1] == "}":
        return " + ".join(reversed(str_pol.split(" + ")))
    
    if eval(is_deg) == deg:
        return str_pol
    else:
        return " + ".join(reversed(str_pol.split(" + ")))



#def penultimate_two_cell_E_latex():

#print("\\begin{tabular}{|l||%s|}"%("|").join(["l"]*n) + " \\hline")
#print(" & " + " & ".join([ "$%d$"%j for j in range(1,n+1) ]) + " \\"+"\\"+" \\hline\\hline")

for i in range(1,n+1):
    print( "$%d$ & "%i + " & ".join( [ "$"+ nice_print(p[join(i,j)]) +"$" for j in range(5,n+1)   ] ) + " \\"+"\\"+" \\hline" )

#print("\\end{tabular}\n")

$1$ & $v^{116} + v^{114} + v^{112} + 2v^{110} + v^{108} + 2v^{106} + 2v^{104} + v^{102} + 2v^{100} + v^{98} + v^{96} + v^{94}$ & $v^{115} + v^{113} + v^{111} + v^{109} + v^{107} + 2v^{105} + v^{103} + v^{101} + v^{99} + v^{97} + v^{95}$ & $v^{114} + v^{112} + v^{108} + v^{106} + v^{104} + v^{102} + v^{98} + v^{96}$ & $v^{113} + v^{107} + v^{103} + v^{97}$ \\ \hline
$2$ & $v^{117} + v^{115} + 2v^{113} + 2v^{111} + 2v^{109} + 3v^{107} + 2v^{105} + 3v^{103} + 2v^{101} + 2v^{99} + 2v^{97} + v^{95} + v^{93}$ & $v^{116} + v^{114} + 2v^{112} + v^{110} + 2v^{108} + 2v^{106} + 2v^{104} + 2v^{102} + v^{100} + 2v^{98} + v^{96} + v^{94}$ & $v^{115} + v^{113} + v^{111} + v^{109} + v^{107} + 2v^{105} + v^{103} + v^{101} + v^{99} + v^{97} + v^{95}$ & $v^{114} + v^{110} + v^{106} + v^{104} + v^{100} + v^{96}$ \\ \hline
$3$ & $v^{117} + 2v^{115} + 2v^{113} + 3v^{111} + 3v^{109} + 3v^{107} + 4v^{105} + 3v^{103} + 3v^{101} + 3v^{99} + 2v^{97} + 2v^{95} + v^{93}$ & $v^{116} + 2v^{114} + 2v^{112} + 2v^{110